In [1]:
# ALI BABOLHAVAEJI
# 7/6/2019

from torchvision import datasets , transforms
from torch.utils.data import *
import torch
import torchvision
import numpy as np
from torch import nn


transform=transforms.Compose([transforms.ToTensor()])
train_data=datasets.MNIST(root='MNIST_dataset',
                          train=True,
                          download=False,
                          transform=transform)

test_data=datasets.MNIST(root='MNIST_dataset',
                         train=False,
                         download=False,
                         transform=transform)

print('MNIST train dataset has {} images.'.format(len(train_data)))
print('MNIST test  dataset has {} images.'.format(len(test_data)))


MNIST train dataset has 60000 images.
MNIST test  dataset has 10000 images.


In [2]:
def split_image_data(train_data,
                     test_data=None,
                     batch_size=20,
                     num_workers=0,
                     valid_size=0.2,
                     sampler=SubsetRandomSampler):
    
    num_train=len(train_data)
    indices= list(range(num_train))
    np.random.shuffle(indices)
    split=int(np.floor(valid_size*num_train))
#     print(split)
    train_idx,valid_idx=indices[split:],indices[:split]
    train_sampler=sampler(train_idx)
    valid_sampler=sampler(valid_idx)
    
    if test_data is not None:
        test_loader= DataLoader(test_data, batch_size=batch_size,
                                num_workers=num_workers) 
    else:
        train_idx,test_idx= train_idx[split:],train_idx[:split]
        train_sampler=sampler(train_idx)
        test_sampler =sampler(test_idx)

        test_loader= DataLoader(train_data, batch_size=batch_size,
                                 num_workers=num_workers,
                                 sampler=test_sampler)
        
    train_loader= DataLoader(train_data, batch_size=batch_size,
                             num_workers=num_workers,
                             sampler=train_sampler)
    
    valid_loader= DataLoader(train_data,batch_size=batch_size,
                             num_workers=num_workers, 
                             sampler=valid_sampler)
    
    return train_loader, valid_loader , test_loader
#     return train_loader , test_loader

In [3]:
# load train data firt for data normalization
batch_size=50
trian_loader= DataLoader(train_data, batch_size=batch_size)
test_loader= DataLoader(test_data, batch_size=batch_size)

In [4]:


idx=iter(trian_loader)
imgs,target=next(idx)
# by this method we can get the first batch whic in each bath we have(50 images)
#based on the batch size
print(imgs.shape ,target)
print(imgs[0].dtype)


# calculate the mean and standard varience of whole dataset

# I m not sure about calculation of total standard deviation of dataset  at following dataset !!!
def calculate_img_stats(loader):
    mean = 0.
    std = 0.
    nb_samples = 0.
    for imgs,_ in loader:
        batch_samples = imgs.size(0)
        mean += (imgs.mean(2).mean(2).sum(0))
        std  += (imgs.std(3).std(2).sum(0))
        nb_samples += batch_samples
    return mean/nb_samples ,std/nb_samples

mean,std=calculate_img_stats(trian_loader)
print(mean,std)

mean_test,std_test=calculate_img_stats(test_loader)
print(mean_test,std_test)


torch.Size([50, 1, 28, 28]) tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5,
        9, 3])
torch.float32
tensor([0.1307]) tensor([0.1632])
tensor([0.1325]) tensor([0.1642])


In [5]:
# data normaliztion and augmentation on train data 

transform=transforms.Compose([transforms.RandomHorizontalFlip(),
                              transforms.RandomRotation(10),
                              transforms.ToTensor(),transforms.Normalize([0.1307],[0.1632])])

train_data=datasets.MNIST(root='MNIST_dataset',
                          train=True,
                          download=False,
                          transform=transform)

# data normalization on the test data

transform=transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize([0.1325],[0.1642])])

test_data=datasets.MNIST(root='MNIST_dataset',
                          train=False,
                          download=False,
                          transform=transform)


In [6]:
# split data to train, test and validation


trian_loader, valid_loader, test_loader= split_image_data(train_data=train_data,
                                                          test_data=test_data,
                                                          valid_size=0.2,
                                                         batch_size=batch_size)
print('Each bache of data has {} images.'.format(batch_size))
print('Train_data:      {}  baches \nValidation data: {}  baches \nTest_data:       {}  baches'.format(len(trian_loader),len(valid_loader),len(test_loader)))

Each bache of data has 50 images.
Train_data:      960  baches 
Validation data: 240  baches 
Test_data:       200  baches


In [138]:
import time as time
from collections import defaultdict
import math


def update_classwise_accuracies(preds,labels,class_correct,class_totals):
    correct = np.squeeze(preds.eq(labels.data.view_as(preds)))
    for i in range(labels.shape[0]):
        label = labels.data[i].item()
        class_correct[label] += correct[i].item()
        class_totals[label] += 1

def get_accuracies(class_names,class_correct,class_totals):

    accuracy = (100*np.sum(list(class_correct.values()))/np.sum(list(class_totals.values())))
    class_accuracies = [(class_names[i],100.0*(class_correct[i]/class_totals[i]))
                        for i in class_names.keys() if class_totals[i] > 0]
    return accuracy,class_accuracies
        
        
        

class Network(nn.Module):
    def __init__(self,device=None):
        super().__init__()
        if device is not None:
                self.device = device
        else:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print('The Device is seted as: {}'.format(self.device))
        self.best_accuracy = 0.
            
    def forward(self,x):
        pass
    
    def get_device(self):
        return(self.device)
        
    def train_(self,trainloader,criterion,optimizer,print_every):
        self.train()
        t0 = time.time()
        batches = 0
        running_loss = 0
        for inputs, labels in trainloader:
            batches += 1
            #t1 = time.time()

            inputs, labels = inputs.to(self.device), labels.to(self.device)
            optimizer.zero_grad()
            outputs = self.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            loss = loss.item()
            #print('training this batch took {:.3f} seconds'.format(time.time() - t1))
            running_loss += loss
            if batches % print_every == 0:
                print(f"{time.asctime()}.."
                        f"Time Elapsed = {time.time()-t0:.3f}.."
                        f"Batch {batches+1}/{len(trainloader)}.. "
                        f"Average Training loss: {running_loss/(batches):.3f}.. "
                        f"Batch Training loss: {loss:.3f}.. "
                        )
                t0 = time.time()
                return running_loss/len(trainloader)
            
    def validate_(self,validloader):
        running_loss = 0.
        accuracy = 0
        class_correct = defaultdict(int)
        class_totals = defaultdict(int)
        self.eval()
        with torch.no_grad():
            for inputs, labels in validloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.forward(inputs)
                loss = self.criterion(outputs, labels)
                running_loss += loss.item()
                _, preds = torch.max(torch.exp(outputs), 1)
                update_classwise_accuracies(preds,labels,class_correct,class_totals)
                accuracy = (100*np.sum(list(class_correct.values()))/np.sum(list(class_totals.values())))
                self.train()
                return (running_loss/len(validloader),accuracy)


    def evaluate(self,testloader):
        self.eval()
        self.model.to(self.device)
        class_correct = defaultdict(int)
        class_totals = defaultdict(int)
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.forward(inputs)
                ps = torch.exp(outputs)
                _, preds = torch.max(ps, 1)
                update_classwise_accuracies(preds,labels,class_correct,class_totals)

        self.train()
        return get_accuracies(self.class_names,class_correct,class_totals)


    def predict(self,inputs,topk=1):
        self.eval()
        self.model.to(self.device)
        with torch.no_grad():
            inputs = inputs.to(self.device)
            outputs = self.forward(inputs)
            ps = torch.exp(outputs)
            p,top = ps.topk(topk, dim=1)
        return p,top

    def fit(self,trainloader,validloader,epochs=2,print_every=10,validate_every=1):

        for epoch in range(epochs):
            self.model.to(self.device)
            print('epoch {:3d}/{}'.format(epoch+1,epochs))
            epoch_train_loss =  self.train_(trainloader,self.criterion,
                                            self.optimizer,print_every)

            if  validate_every and (epoch % validate_every == 0):
                t2 = time.time()
                epoch_validation_loss,epoch_accuracy = self.validate_(validloader)
                time_elapsed = time.time() - t2
                print(f"{time.asctime()}--Validation time {time_elapsed:.3f} seconds.."
                      f"Epoch {epoch+1}/{epochs}.. "
                      f"Epoch Training loss: {epoch_train_loss:.3f}.. "
                      f"Epoch validation loss: {epoch_validation_loss:.3f}.. "
                      f"validation accuracy: {epoch_accuracy:.3f}")
                if self.best_accuracy == 0. or (epoch_accuracy > self.best_accuracy):
                    print('updating best accuracy: previous best = {:.3f} new best = {:.3f}'.format(self.best_accuracy,
                                                                                     epoch_accuracy))
                self.best_accuracy = epoch_accuracy
                torch.save(self.state_dict(),self.best_accuracy_file)

                self.train() # just in case we forgot to put the model back to train mode in validate

        print('loading best accuracy model')
        self.load_state_dict(torch.load(self.best_accuracy_file))





    def set_criterion(self,criterion_name):
        if criterion_name.lower() == 'nllloss':
            self.criterion_name = 'NLLLoss'
            self.criterion = nn.NLLLoss()
        elif criterion_name.lower() == 'crossentropyloss':
            self.criterion_name = 'CrossEntropyLoss'
            self.criterion = nn.CrossEntropyLoss()

    def set_optimizer(self,params,optimizer_name='adam',lr=0.003):
        from torch import optim

        if optimizer_name.lower() == 'adam':
            print('setting optim Adam')
            self.optimizer = optim.Adam(params,lr=lr)
            self.optimizer_name = optimizer_name
        elif optimizer.lower() == 'sgd':
            print('setting optim SGD')
            self.optimizer = optim.SGD(params,lr=lr)

        elif optimizer.lower() == 'adadelta':
            print('setting optim Ada Delta')
            self.optimizer = optim.Adadelta(params)

    def set_model_params(self,
                         criterion_name,
                         optimizer_name,
                         lr, # learning rate
                         dropout_p,
                         model_name,
                         best_accuracy,
                         best_accuracy_file,
                         class_names):

        self.set_criterion(criterion_name)
        self.set_optimizer(self.parameters(),optimizer_name,lr=lr)
        self.lr = lr
        self.dropout_p = dropout_p
        self.model_name =  model_name
        self.best_accuracy = best_accuracy
        self.best_accuracy_file = best_accuracy_file
        self.class_names = class_names

    def get_model_params(self):
        params = {}
        params['device'] = self.device
#         params['model_name'] = self.model_name
        params['optimizer_name'] = self.optimizer_name
        params['criterion_name'] = self.criterion_name
        params['lr'] = self.lr
        params['dropout_p'] = self.dropout_p
        params['best_accuracy'] = self.best_accuracy
        params['best_accuracy_file'] = self.best_accuracy_file
        params['class_names'] = self.class_names
        return params

    

In [89]:
A1=Network()


The Device is seted as: cuda


In [140]:
def flatten_tensor(x):
    return x.view(x.shape[0],-1)

class FC(Network):
    def __init__(self,num_inputs,
                 num_outputs,
                 layers=[],
                 lr=0.003,
                 class_names=None,
                 optimizer_name='Adam',
                 dropout_p=0.2,
                 non_linearity='relu',
                 criterion_name='NLLLoss',
                 model_type='classifier',
                 best_accuracy=0.,
                 best_accuracy_file ='best_accuracy.pth',
                 chkpoint_file ='chkpoint_file.pth',
                 device=None):

        super().__init__(device=device)
        self.get_model_params()
        
        self.set_model_params(criterion_name,
                              optimizer_name,
                              lr,
                              dropout_p,
                              'FC',
                              best_accuracy,
                              best_accuracy_file,
                              chkpoint_file
                              )
        self.non_linearity = non_linearity
        self.model = nn.Sequential()
        if len(layers) > 0:
            self.model.add_module('fc1',nn.Linear(num_inputs,layers[0]))
            self.model.add_module('relu1',nn.ReLU())
            self.model.add_module('dropout1',nn.Dropout(p=dropout_p,inplace=True))

            for i in range(1,len(layers)):
                self.model.add_module('fc'+str(i+1),nn.Linear(layers[i-1],layers[i]))
                self.model.add_module('relu'+str(i+1),nn.ReLU())
                self.model.add_module('dropout'+str(i+1),nn.Dropout(p=dropout_p))
                self.model.add_module('out',nn.Linear(layers[-1],num_outputs))

        else:
            self.model.add_module('out',nn.Linear(num_inputs,num_outputs))
            if model_type.lower() == 'classifier' and criterion_name.lower() == 'nllloss':
                self.model.add_module('logsoftmax',nn.LogSoftmax(dim=1))
                self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.layer_dims = layers
        if class_names is not None:
            self.class_names = class_names
        else:
            self.class_names = {str(k):v for k,v in enumerate(list(range(num_outputs)))}
            def forward(self,x):
                return self.model(flatten_tensor(x))
            
    def _get_dropout(self):
        for layer in self.model:
            if type(layer) == torch.nn.modules.dropout.Dropout:
                return layer.p

    def _set_dropout(self,p=0.2):
        for layer in self.model:
            if type(layer) == torch.nn.modules.dropout.Dropout:
                print('FC: setting dropout prob to {:.3f}'.format(p))
                layer.p=p
                
    def set_model_params(self,
                         criterion_name,
                         optimizer_name,
                         lr,
                         dropout_p,
                         model_name,
                         model_type,
                         best_accuracy,
                         best_accuracy_file,
                         chkpoint_file,
                         num_inputs,
                         num_outputs,
                         layers,
                         class_names):
        super(FC, self).set_model_params(criterion_name,
                              optimizer_name,
                              lr,
                              dropout_p,
                              model_name,
                              best_accuracy,
                              best_accuracy_file,
                              chkpoint_file
                              )

        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.layer_dims = layers
        self.model_type = model_type

        if class_names is not None:
            self.class_names = class_names
        else:
            self.class_names = {k:str(v) for k,v in enumerate(list(range(num_outputs)))}

    def get_model_params(self):
        params = super(FC, self).get_model_params()
        params['num_inputs'] = self.num_inputs
        params['num_outputs'] = self.num_outputs
        params['layers'] = self.layer_dims
        params['model_type'] = self.model_type
        params['class_names'] = self.class_names
        params['device'] = self.device
        return params
    
    def load_chkpoint(chkpoint_file):

        restored_data = torch.load(chkpoint_file)

        params = restored_data['params']
        print('load_chkpoint: best accuracy = {:.3f}'.format(params['best_accuracy']))

        if params['model_type'].lower() == 'classifier':
            net = FC( num_inputs=params['num_inputs'],
                      num_outputs=params['num_outputs'],
                      layers=params['layers'],
                      device=params['device'],
                      criterion_name = params['criterion_name'],
                      optimizer_name = params['optimizer_name'],
#                       model_name = params['model_name'],
                      lr = params['lr'],
                      dropout_p = params['dropout_p'],
                      best_accuracy = params['best_accuracy'],
                      best_accuracy_file = params['best_accuracy_file'],
                      chkpoint_file = params['chkpoint_file'],
                      class_names =  params['class_names']
              )

        net.load_state_dict(torch.load(params['best_accuracy_file']))

        net.to(params['device'])

        return net

        

In [141]:
net =  FC(num_inputs=784,
          num_outputs=10,
          layers=[512,512],
          optimizer_name='Adadelta',
          best_accuracy_file ='best_accuracy_mnist_fc_test.pth',
          chkpoint_file='chkpoint_file_mnist_fc_test.pth')


The Device is seted as: cuda


AttributeError: 'FC' object has no attribute 'optimizer_name'